# Text Classification, Topic Modelling using Neural Network, for Academic Article

## Table of Content
1. [Text Classification](#1)
    - 1.1 [Logistic Regression](#1.1)
    - 1.2 [Recurrent Neural Network](#1.2)
2. [Topic Modelling](#2)

## Text Classification <a class="anchor" id="1"></a>

### Logistic Regression <a class="anchor" id="1.1"></a>

### Recurrent Neural Network <a class="anchor" id="1.2"></a>

## Topic Modelling <a class="anchor" id="2"></a>